# DicomStructureFile Class Demonstration

This notebook demonstrates the functionality of the `DicomStructureFile` class using a test DICOM RT Structure file.

## Overview
The `DicomStructureFile` class provides a convenient interface for:
- Loading DICOM RT Structure files
- Extracting structure information and metadata
- Reading contour sequences and converting them to ContourPoints
- Converting contour data to pandas DataFrames for analysis

## 1. Import Required Libraries

First, let's import the necessary libraries including our custom DicomStructureFile class.

In [37]:
# Import required libraries
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add the src directory to the Python path
sys.path.append('src')

# Import our custom DicomStructureFile class
from dicom import DicomStructureFile

# Import related classes
from types_and_classes import SliceIndexType
from contours import ContourPoints
from structure_set import StructureSet

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load the DICOM Structure File

Now let's load the test DICOM RT Structure file using the DicomStructureFile class.

The DICOM file contains the following structures:

![MultiVolume_A](<../Images/MultiVolume_A.png>)


In [38]:
# Define the path to the test DICOM file
test_file_name = "RS.GJS_Struct_Tests.MultiVolume_A (1).dcm"
tests_dir = Path.cwd() / r'Tests'
tests_dir = tests_dir.resolve()

In [39]:
list(tests_dir.glob('*.dcm'))

[PosixPath('/workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.MultiVolume_A (2).dcm'),
 PosixPath('/workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.Relations.dcm'),
 PosixPath('/workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.MultiVolume_A (1).dcm'),
 PosixPath('/workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.MultiVolume_A.dcm')]

In [40]:

# Load the DICOM structure file using file_name parameter
dicom_file = DicomStructureFile(
    top_dir=tests_dir,
    file_name=test_file_name
)

print(f"Successfully loaded: {dicom_file}")
print(f"File path: {dicom_file.file_path}")
print(f"Is RT Structure file: {dicom_file.is_structure_file()}")

INFO:dicom:Successfully loaded DICOM dataset from RS.GJS_Struct_Tests.MultiVolume_A (1).dcm
INFO:dicom:Extracted 730 contours from 7 ROIs


Successfully loaded: DICOM RT Structure: MultiVolume_A for patient GJS_Struct_Tests
File path: /workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.MultiVolume_A (1).dcm
Is RT Structure file: True


## 3. Explore Structure File Properties

Let's examine the basic properties and metadata of the loaded DICOM structure file.

In [41]:
# Get structure file information
structure_info = dicom_file.get_structure_set_info()

print("=== DICOM Structure File Information ===")
for key, value in structure_info.items():
    print(f"{key}: {value}")

# Access some basic DICOM dataset attributes
dataset = dicom_file.dataset
print("\n=== Additional DICOM Attributes ===")
print(f"Modality: {getattr(dataset, 'Modality', 'Not available')}")
print(f"Study Date: {getattr(dataset, 'StudyDate', 'Not available')}")
print(f"Manufacturer: {getattr(dataset, 'Manufacturer', 'Not available')}")
print(f"Software Version: {getattr(dataset, 'SoftwareVersions', 'Not available')}")

# Check if ROI contour sequence exists
if hasattr(dataset, 'ROIContourSequence'):
    print(f"\nNumber of ROI Contours: {len(dataset.ROIContourSequence)}")
else:
    print("\nNo ROI Contour Sequence found")

=== DICOM Structure File Information ===
PatientName: StructureVolumes^Test
PatientLastName: StructureVolumes
PatientID: GJS_Struct_Tests
StructureSet: MultiVolume_A
StudyID: Phantom2
SeriesNumber: 9
File: /workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.MultiVolume_A (1).dcm

=== Additional DICOM Attributes ===
Modality: RTSTRUCT
Study Date: 20240218
Manufacturer: Varian Medical Systems
Software Version: 4.2.7.0

Number of ROI Contours: 7


## 4. Access Structure Set Information

Let's examine the structure set and ROI information in more detail.

In [42]:
# Examine the Structure Set ROI Sequence
if hasattr(dataset, 'StructureSetROISequence'):
    print("=== Structure Set ROI Information ===")
    roi_info = []
    
    for i, roi in enumerate(dataset.StructureSetROISequence):
        roi_data = {
            'ROI Number': getattr(roi, 'ROINumber', 'N/A'),
            'ROI Name': getattr(roi, 'ROIName', 'N/A'),
            'ROI Generation Algorithm': getattr(roi, 'ROIGenerationAlgorithm', 'N/A'),
            'Referenced Frame of Reference': getattr(roi, 'ReferencedFrameOfReferenceUID', 'N/A')
        }
        roi_info.append(roi_data)
        
        print(f"\nROI {i+1}:")
        for key, value in roi_data.items():
            print(f"  {key}: {value}")

    # Convert to DataFrame for easier viewing
    roi_df = pd.DataFrame(roi_info)
    print("\n=== ROI Summary Table ===")
    print(roi_df.to_string(index=False))
else:
    print("No Structure Set ROI Sequence found")

=== Structure Set ROI Information ===

ROI 1:
  ROI Number: 1
  ROI Name: BODY
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 2:
  ROI Number: 7
  ROI Name: OverlappedSphere
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 3:
  ROI Number: 8
  ROI Name: OverlappedShells
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 4:
  ROI Number: 9
  ROI Name: Surrounds
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 5:
  ROI Number: 10
  ROI Name: Embeds
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 6:
  ROI Number: 13
  ROI Name: PartialEmbeded
  ROI Generation Algorithm: MANUAL
  Referenced Frame

## 5. Extract ROI Contour Data

Now let's use our custom method to extract contour data and convert it to ContourPoints objects.

In [43]:
# Extract contour sequences using our custom method
contour_points = dicom_file.get_contour_points()

print(f"=== Contour Extraction Results ===")
print(f"Total ContourPoints objects created: {len(contour_points)}")

# Examine the first few contour points
if contour_points:
    print(f"\n=== Sample ContourPoints Objects ===")
    for i, cp in enumerate(contour_points[:5]):  # Show first 5
        print(f"\nContourPoint {i+1}:")
        print(f"  ROI Number: {cp['ROI']}")
        print(f"  Slice Index: {cp['Slice']}")
        print(f"  Number of Points: {len(cp['Points']) if cp['Points'] else 0}")
        if cp['Points'] and len(cp['Points']) > 0:
            print(f"  First point: {cp['Points'][0]}")
            print(f"  Last point: {cp['Points'][-1]}")

    # Show statistics by ROI
    roi_counts = {}
    for cp in contour_points:
        roi_num = cp['ROI']
        if roi_num not in roi_counts:
            roi_counts[roi_num] = {'slices': 0, 'total_points': 0}
        roi_counts[roi_num]['slices'] += 1
        if cp['Points']:
            roi_counts[roi_num]['total_points'] += len(cp['Points'])
    
    print(f"\n=== Contour Statistics by ROI ===")
    for roi_num, stats in roi_counts.items():
        print(f"ROI {roi_num}: {stats['slices']} slices, {stats['total_points']} total points")
        
else:
    print("No contour points found in the dataset")

INFO:dicom:Extracted 730 contours from 7 ROIs


=== Contour Extraction Results ===
Total ContourPoints objects created: 730

=== Sample ContourPoints Objects ===

ContourPoint 1:
  ROI Number: 1
  Slice Index: -10.0
  Number of Points: 680
  First point: [ -9.902  -9.906 -10.   ]
  Last point: [ -9.906  -9.902 -10.   ]

ContourPoint 2:
  ROI Number: 1
  Slice Index: -9.9
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.9  ]
  Last point: [-10.001  -9.902  -9.9  ]

ContourPoint 3:
  ROI Number: 1
  Slice Index: -9.8
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.8  ]
  Last point: [-10.001  -9.902  -9.8  ]

ContourPoint 4:
  ROI Number: 1
  Slice Index: -9.7
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.7  ]
  Last point: [-10.001  -9.902  -9.7  ]

ContourPoint 5:
  ROI Number: 1
  Slice Index: -9.6
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.6  ]
  Last point: [-10.001  -9.902  -9.6  ]

=== Contour Statistics by ROI ===
ROI 1: 201 slices, 136680 total points
ROI 7: 98 slices, 133

## Summary

This notebook has demonstrated the complete functionality of the `DicomStructureFile` class:

1. **Loading DICOM Files**: Successfully loaded a DICOM RT Structure file using either file_name or file_path parameters
2. **Metadata Extraction**: Retrieved comprehensive structure file information and ROI details
3. **Contour Processing**: Extracted all contour sequences and converted them to `ContourPoints` objects

### Key Features Demonstrated:
- Flexible initialization with multiple parameter options
- Robust error handling and validation
- Integration with existing `ContourPoints` and related classes
- Comprehensive metadata extraction
- Multiple export format support

### Next Steps:
- The `DicomStructureFile` class can be integrated into larger workflow pipelines


## 9. Integration with StructureSet Class
Now let's demonstrate how to use the DicomStructureFile with the StructureSet class to analyze structure relationships.


In [44]:
print("=== Creating StructureSet from DicomStructureFile ===")

# Method 1: Using DicomStructureFile (recommended approach)
structure_set = StructureSet(dicom_structure_file=dicom_file)

print(f"Successfully created StructureSet with {len(structure_set.structures)} structures")


INFO:structure_set:Building StructureSet from 730 contour points


=== Creating StructureSet from DicomStructureFile ===
Successfully created StructureSet with 7 structures


In [45]:
# Display the structures that were loaded
print("\n=== Loaded Structures ===")
for roi, structure in structure_set.structures.items():
    print(f"ROI {roi}: {structure.name}")



=== Loaded Structures ===
ROI 1: BODY
ROI 7: OverlappedSphere
ROI 8: OverlappedShells
ROI 9: Surrounds
ROI 10: Embeds
ROI 12: PartialSurround
ROI 13: PartialEmbeded


In [46]:
# Get basic structure information
print("=== Structure Summary ===")
structure_summary = structure_set.summary()
print(structure_summary.to_string(index=False))


=== Structure Summary ===
 ROI             Name  Physical_Volume  Exterior_Volume  Hull_Volume  Num_Contours  Num_Slices
   1             BODY      8599.789625      8599.789625  8599.789625           217         217
   7 OverlappedSphere       136.207300       136.207300   136.363853           106          53
   8 OverlappedShells        70.311990       137.761726   138.123647           194          55
   9        Surrounds       266.403816       302.004948   302.618143           137          90
  10           Embeds        14.402917        14.402917    14.418471            33          33
  12  PartialSurround       149.078283       149.078283   165.721736            92          90
  13   PartialEmbeded         3.359495         3.359495     3.364644            29          29


In [47]:
# Calculate relationships between all structures
print("\n=== Calculating Structure Relationships ===")
structure_set.calculate_relationships()

print(f"Total relationships calculated: {structure_set.relationship_graph.number_of_edges()}")



=== Calculating Structure Relationships ===


Total relationships calculated: 21


In [48]:
# Display the relationship summary table
print("\n=== Structure Relationship Summary Table ===")
relationship_summary = structure_set.relationship_summary()
relationship_summary


=== Structure Relationship Summary Table ===


Structure_B,BODY,OverlappedSphere,OverlappedShells,Surrounds,Embeds,PartialSurround,PartialEmbeded
Structure_A,,,,,,,
BODY,Equals,Relationship: Contains,Relationship: Contains,Relationship: Contains,Relationship: Contains,Relationship: Contains,Relationship: Contains
OverlappedSphere,None,Equals,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint
OverlappedShells,None,Relationship: Disjoint,Equals,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint
Surrounds,None,Relationship: Disjoint,Relationship: Disjoint,Equals,Relationship: Surrounds,Relationship: Disjoint,Relationship: Disjoint
Embeds,None,Relationship: Disjoint,Relationship: Disjoint,None,Equals,Relationship: Disjoint,Relationship: Disjoint
PartialSurround,None,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,Equals,Relationship: Shelters
PartialEmbeded,None,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,Relationship: Disjoint,None,Equals



![MultiVolume_A](<../Images/MultiVolume_A.png>)

In [49]:
# Demonstrate structure filtering with exclusions
print("\n=== Structure Filtering Example ===")

# Load the DICOM structure file using file_name parameter
filtered_dicom_file = DicomStructureFile(
    top_dir=tests_dir,
    file_name=test_file_name
)

# Apply exclusions to remove structures starting with common prefixes
filtered_dicom_file.filter_exclusions(
    exclude_prefixes=['BODY', 'External'], 
    case_sensitive=False,
    exclude_empty=True)

# Create a new StructureSet with exclusions applied
filtered_structure_set = StructureSet(dicom_structure_file=filtered_dicom_file)

print(f"Structures before filtering: {len(structure_set.structures)}")
print(f"Structures after filtering: {len(filtered_structure_set.structures)}")

if len(filtered_structure_set.structures) > 0:
    print("\n=== Remaining Structures After Filtering ===")
    for roi, structure in filtered_structure_set.structures.items():
        print(f"ROI {roi}: {structure.name}")
    
    # Calculate relationships for the filtered set
    filtered_structure_set.calculate_relationships()
    
    if filtered_structure_set.relationship_graph.number_of_edges() > 0:
        print("\n=== Filtered Relationship Summary ===")
        filtered_relationships = filtered_structure_set.relationship_summary()
        print(filtered_relationships)
else:
    print("All structures were filtered out.")


INFO:dicom:Successfully loaded DICOM dataset from RS.GJS_Struct_Tests.MultiVolume_A (1).dcm



=== Structure Filtering Example ===


INFO:dicom:Extracted 730 contours from 7 ROIs
INFO:dicom:Filtered 201 contours from 1 excluded ROIs. Remaining: 529 contours from 6 ROIs
INFO:structure_set:Building StructureSet from 529 contour points


Structures before filtering: 7
Structures after filtering: 6

=== Remaining Structures After Filtering ===
ROI 7: OverlappedSphere
ROI 8: OverlappedShells
ROI 9: Surrounds
ROI 10: Embeds
ROI 12: PartialSurround
ROI 13: PartialEmbeded

=== Filtered Relationship Summary ===
Structure_B             OverlappedSphere        OverlappedShells  \
Structure_A                                                        
OverlappedSphere                  Equals  Relationship: Disjoint   
OverlappedShells  Relationship: Disjoint                  Equals   
Surrounds         Relationship: Disjoint  Relationship: Disjoint   
Embeds            Relationship: Disjoint  Relationship: Disjoint   
PartialSurround   Relationship: Disjoint  Relationship: Disjoint   
PartialEmbeded    Relationship: Disjoint  Relationship: Disjoint   

Structure_B                    Surrounds                   Embeds  \
Structure_A                                                         
OverlappedSphere  Relationship: Disjoint   R

In [50]:
# Show individual structure details
print("\n=== Individual Structure Details ===")

for roi, structure in structure_set.structures.items():
    print(f"\n--- Structure ROI {roi}: {structure.name} ---")
    print(f"Physical Volume: {structure.physical_volume:.3f} cc")
    print(f"Hull Volume: {structure.hull_volume:.3f} cc")
    print(f"Number of Slices: {len(structure.region_table)}")
    print(f"Number of Contours: {len(structure.contour_graph)}")
    
    # Show slice range
    if hasattr(structure, 'slice_sequence') and structure.slice_sequence:
        slice_indices = list(structure.slice_sequence.slice_indices)
        if slice_indices:
            print(f"Slice Range: {min(slice_indices):.1f} to {max(slice_indices):.1f}")



=== Individual Structure Details ===

--- Structure ROI 1: BODY ---
Physical Volume: 8599.790 cc
Hull Volume: 8599.790 cc
Number of Slices: 217
Number of Contours: 217

--- Structure ROI 7: OverlappedSphere ---
Physical Volume: 136.207 cc
Hull Volume: 136.364 cc
Number of Slices: 53
Number of Contours: 106

--- Structure ROI 8: OverlappedShells ---
Physical Volume: 70.312 cc
Hull Volume: 138.124 cc
Number of Slices: 55
Number of Contours: 194

--- Structure ROI 9: Surrounds ---
Physical Volume: 266.404 cc
Hull Volume: 302.618 cc
Number of Slices: 90
Number of Contours: 137

--- Structure ROI 10: Embeds ---
Physical Volume: 14.403 cc
Hull Volume: 14.418 cc
Number of Slices: 33
Number of Contours: 33

--- Structure ROI 12: PartialSurround ---
Physical Volume: 149.078 cc
Hull Volume: 165.722 cc
Number of Slices: 90
Number of Contours: 92

--- Structure ROI 13: PartialEmbeded ---
Physical Volume: 3.359 cc
Hull Volume: 3.365 cc
Number of Slices: 29
Number of Contours: 29


In [51]:
# Create a comprehensive analysis report
print("\n=== Comprehensive Analysis Report ===")

analysis_report = {
    'File_Info': dicom_file.get_structure_set_info(),
    'Structure_Count': len(structure_set.structures),
    'Relationship_Count': structure_set.relationship_graph.number_of_edges(),
    'Total_Contours': len(dicom_file.contour_points) if dicom_file.contour_points else 0
}

print("=== DICOM File Analysis Report ===")
for key, value in analysis_report.items():
    if isinstance(value, dict):
        print(f"\n{key.replace('_', ' ')}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"{key.replace('_', ' ')}: {value}")



=== Comprehensive Analysis Report ===
=== DICOM File Analysis Report ===

File Info:
  PatientName: StructureVolumes^Test
  PatientLastName: StructureVolumes
  PatientID: GJS_Struct_Tests
  StructureSet: MultiVolume_A
  StudyID: Phantom2
  SeriesNumber: 9
  File: /workspaces/StructureRelations/Tests/RS.GJS_Struct_Tests.MultiVolume_A (1).dcm
Structure Count: 7
Relationship Count: 21
Total Contours: 730


## Summary of Integration

This extended demonstration shows how the `DicomStructureFile` class integrates seamlessly with the `StructureSet` class to provide:

1. **Automatic Structure Loading**: The `StructureSet` can be initialized directly from a `DicomStructureFile`
2. **Relationship Analysis**: Calculate spatial relationships between all structures
3. **Structure Filtering**: Apply exclusion patterns to remove unwanted structures
4. **Comprehensive Reporting**: Generate detailed reports on structure properties and relationships
5. **Data Export**: Export relationship data and analysis reports to various formats

### Key Benefits:
- **Streamlined Workflow**: Single initialization creates both contour data and structure relationships
- **Flexible Filtering**: Remove common utility structures (BODY, External, etc.) before analysis
- **Rich Metadata**: Access both DICOM metadata and calculated structure properties
- **Export Capabilities**: Generate reports and data files for further analysis

This integration demonstrates the power of combining DICOM file handling with advanced spatial relationship analysis.